# 要約 
このJupyter Notebookは、Kaggleの「LLM 20 Questions」コンペティションに参加するためのエージェントを構築・実行することを目的としています。具体的には、質問者エージェントと回答者エージェントを用いて「20の質問」ゲームをプレイする際のロジックを実装しています。

### 取り組んでいる問題
- ゲーム「20の質問」において、質問者エージェントが効果的に質問を行い、回答者エージェントが「はい」または「いいえ」という形で応答することで、ターゲットとなる単語をできるだけ少ない質問で推測するための処理を行います。

### 手法とライブラリ
1. **TPU/GPU環境の設定**: 
   - TPUを使用するための設定が行われており、TPUデバイスに合わせたテンソル設定が施されています。
   - `pytorch-xla`に関連するセットアップが行われており、`torch`ライブラリが利用されています。

2. **Gemma モデルの利用**:
   - `gemma_pytorch`というリポジトリからGemmaモデルの重みや設定を取得し、使用します。このモデルは因果生成モデルとして動作します。
   - `GemmaForCausalLM`クラスを用いて、特定のコンフィギュレーションに基づいた言語モデルを初期化します。

3. **エージェントの実装**:
   - 質問者エージェント (`GemmaQuestionerAgent`) と回答者エージェント (`GemmaAnswererAgent`) がそれぞれ独自のロジックを持って実装され、ユーザーのターンを管理します。
   - 質問と回答をインタリーブする機能や、ユーザーの入力をテンプレートに基づいてフォーマットするための `GemmaFormatter` クラスが定義されています。

4. **環境の構築とゲームプレイ**:
   - Kaggleの環境 (`kaggle_environments`)を使用して、ゲームをシミュレーションし、実際にエージェントが相互にプレイすることが可能な設定となっています。

5. **提出とパッケージング**:
   - 最終的には、生成されたエージェントと関連データを圧縮ファイルとして提出する準備が整えられています。

このノートブックは、Kaggleでの限られた質問数での推測問題に対する戦略的かつ効率的なアプローチを示しているとともに、高度な自然言語処理能力を活かしたAIエージェントの設計を反映しています。

---


# 用語概説 
以下に、Jupyter Notebookの内容に基づいて、機械学習や深層学習に関連して初心者がつまずきそうな専門用語の簡単な解説を列挙します。

### 専門用語と解説

1. **ImmutableDict**
   - 変更できない辞書のデータ構造を提供するライブラリで、主に不変性を保証するために使用される。プログラム中で意図しない変更を防ぎ、データの信頼性を高める。

2. **SentencePiece**
   - 文を単語単位でなく、サブワード単位で分割するためのトークナイザー。これは特に自然言語処理において、未知の単語の処理や多言語対応に役立つ。言語モデルの訓練データを効率的に扱うために使われる。

3. **TPU (Tensor Processing Unit)**
   - Googleが設計したハードウェアアクセラレータで、特に機械学習に最適化されている。TPUは高い計算性能を持ち、特に深層学習モデルの訓練や推論の高速化に利用される。

4. **xm.xla_device()**
   - XLA（Accelerated Linear Algebra）は、TensorFlowの最適化コンパイラで、TPUやGPUで実行するためのデバイスを指定する際に用いる。`xm`はPyTorch XLAライブラリで、TPUをサポートするために使われる。

5. **Causal Language Model (CLM)**
   - 過去のトークン（単語やサブワード）を基に次のトークンを予測するモデル。このアプローチは、生成タスク（文章生成や対話生成など）において効果的。

6. **Few-shot Learning (FSL)**
   - 限られた訓練データ（例: 数個のサンプルのみ）から学習する手法。特に、モデルが新しいタスクに迅速に適応するために利用される。

7. **Interleave**
   - 2つのシーケンス（リストなど）の要素を交互に結びつける処理を指す。特に、質問と回答のペアや、異なるエピソードのデータを混ぜ合わせる際に用いられる。

8. **Top-k Sampling**
   - 次の単語を生成する際に、最も確率の高いk個の単語から選ぶ手法。これにより、生成される文章の多様性を向上させる。

9. **Top-p (nucleus) Sampling**
   - 確率の合計がpになるまでの最小のトークンの集合から単語を選ぶ手法。この手法は、生成されたテキストの品質を向上させるために使用される。

10. **Observations (obs)**
    - エージェントが受け取る情報や状態を指しており、質問、回答、ターンの種類などが含まれるデータ構造。

11. **Context Manager**
    - Pythonの構文で、特定のリソースを管理するための便利な機能。コードのブロックが終了するときにリソースを自動的に解放し、エラー処理を簡素化するために使用される。

12. **Regex (Regular Expressions)**
    - 文字列パターンを調べるために使用される強力なツール。特定の形式に合致する文字列の検索や抽出、置換を行う際に使われる。

以上の専門用語の解説が、初心者がつまずく可能性のあるポイントを理解する助けになることを願っています。

---


<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
%%bash
cd /kaggle/working
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece
git clone https://github.com/google/gemma_pytorch.git > /dev/null
mkdir /kaggle/working/submission/lib/gemma/
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/
```

</div>
<div class="column-right">

# 日本語訳

```python
%%bash
# 現在の作業ディレクトリに移動
cd /kaggle/working
# immutabledict と sentencepiece パッケージをインストール
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece
# gemma_pytorch リポジトリをクローン
git clone https://github.com/google/gemma_pytorch.git > /dev/null
# gemma のディレクトリを作成
mkdir /kaggle/working/submission/lib/gemma/
# gemma_pytorch から必要なファイルを移動
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/
```

</div>
</details>

In [ ]:
%%bash
# 現在の作業ディレクトリに移動
cd /kaggle/working
# immutabledict と sentencepiece パッケージをインストール
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece
# gemma_pytorch リポジトリをクローン
git clone https://github.com/google/gemma_pytorch.git > /dev/null
# gemma のディレクトリを作成
mkdir /kaggle/working/submission/lib/gemma/
# gemma_pytorch から必要なファイルを移動
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# for TPU
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev
```

</div>
<div class="column-right">

# 日本語訳

```python
# TPU用のセットアップ
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# 必要なAPTパッケージをインストール
!python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev
```

</div>
</details>

In [ ]:
# TPU用のセットアップ
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# 必要なAPTパッケージをインストール
!python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# for GPU
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# for TPU
device = xm.xla_device()
torch.set_default_tensor_type('torch.FloatTensor')
```

</div>
<div class="column-right">

# 日本語訳

```python
# GPU用のデバイス設定
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# TPU用のデバイス設定
device = xm.xla_device()
# デフォルトのテンソルタイプを設定
torch.set_default_tensor_type('torch.FloatTensor')
```

</div>
</details>

In [ ]:
# GPU用のデバイス設定
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# TPU用のデバイス設定
device = xm.xla_device()
# デフォルトのテンソルタイプを設定
torch.set_default_tensor_type('torch.FloatTensor')

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
%%writefile submission/main.py

import torch, itertools, contextlib
import os, sys, re
from typing import Iterable
from pathlib import Path

KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib'))
    WEIGHTS_PATH = os.path.join(KAGGLE_AGENT_PATH, "gemma/pytorch/2b-it/2")
else:
    sys.path.insert(0, "/kaggle/working/submission/lib")
    WEIGHTS_PATH = "/kaggle/input/gemma/pytorch/2b-it/2"

from gemma.config import get_config_for_2b
from gemma.model import GemmaForCausalLM

class GemmaFormatter:
    _start_token = '<start_of_turn>'
    _end_token = '<end_of_turn>'
    def __init__(self, sp: str = None, fse: Iterable = None):
        self._system_prompt = sp
        self._few_shot_examples = fse
        self._turn_user = f"{self._start_token}user\n{{}}{self._end_token}\n"
        self._turn_model = f"{self._start_token}model\n{{}}{self._end_token}\n"
        self.reset()
    def __repr__(self):
        return self._state
    def user(self, prompt):
        self._state += self._turn_user.format(prompt)
        return self
    def model(self, prompt):
        self._state += self._turn_model.format(prompt)
        return self
    def start_user_turn(self):
        self._state += f"{self._start_token}user\n"
        return self
    def start_model_turn(self):
        self._state += f"{self._start_token}model\n"
        return self
    def end_turn(self):
        self._state += f"{self._end_token}\n"
        return self
    def reset(self):
        self._state = ""
        if self._system_prompt is not None:
            self.user(self._system_prompt)
        if self._few_shot_examples is not None:
            self.apply_turns(self._few_shot_examples, start_agent='user')
        return self
    def apply_turns(self, turns: Iterable, start_agent: str):
        formatters = [self.model, self.user] if start_agent == 'model' else [self.user, self.model]
        formatters = itertools.cycle(formatters)
        for fmt, turn in zip(formatters, turns):
            fmt(turn)
        return self

@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
    torch.set_default_dtype(dtype)
    yield
    torch.set_default_dtype(torch.float)

class GemmaAgent:
    def __init__(self, sp=None, fse=None):
        self._device = xm.xla_device()
        self.formatter = GemmaFormatter(sp=sp, fse=fse)
        print("Initializing model")
        model_config = get_config_for_2b()
        model_config.tokenizer = WEIGHTS_PATH + '/tokenizer.model'
        with _set_default_tensor_type(model_config.get_dtype()):
            model = GemmaForCausalLM(model_config)
            model.load_weights(WEIGHTS_PATH + '/gemma-2b-it.ckpt')
            self.model = model.to(self._device).eval()
    def __call__(self, obs, *args):
        self._start_session(obs)
        prompt = str(self.formatter)
        response = self._call_llm(prompt)
        response = self._parse_response(response, obs)
        print(f"{response=}")
        return response
    def _start_session(self, obs: dict):
        raise NotImplementedError
    def _call_llm(self, prompt, max_nt=40, **sampler_kwargs):
        if sampler_kwargs is None:
            sampler_kwargs = {'temperature': 0.8, 'top_p': 0.9, 'top_k': 60,}
        response = self.model.generate(
            prompt, device=self._device, output_len=max_nt, **sampler_kwargs,)
        return response
    def _parse_keyword(self, response: str):
        match = re.search(r"(?<=\*\*)([^*]+)(?=\*\*)", response)
        if match is None: keyword = ''
        else: keyword = match.group().lower()
        return keyword
    def _parse_response(self, response: str, obs: dict):
        raise NotImplementedError

def interleave_unequal(x, y):
    return [item for pair in itertools.zip_longest(x, y) for item in pair if item is not None]

class GemmaQuestionerAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    def _start_session(self, obs):
        self.formatter.reset()
        self.formatter.user("Let's play 20 Questions. You are playing the role of the Questioner.")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='model')
        if obs.turnType == 'ask':
            self.formatter.user("Please ask a yes-or-no question.")
        elif obs.turnType == 'guess':
            self.formatter.user("Now guess the keyword. Surround your guess with double asterisks.")
        self.formatter.start_model_turn()
    def _parse_response(self, response: str, obs: dict):
        if obs.turnType == 'ask':
            match = re.search(".+?\?", response.replace('*', ''))
            if match is None: question = "Is it a person?" #make random choice for person, place, thing
            else: question = match.group()
            return question
        elif obs.turnType == 'guess':
            guess = self._parse_keyword(response)
            return guess
        else: raise ValueError("Unknown turn type:", obs.turnType)

class GemmaAnswererAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    def _start_session(self, obs):
        self.formatter.reset()
        self.formatter.user(f"Let's play 20 Questions. You are playing the role of the Answerer. The keyword is {obs.keyword} in the category {obs.category}.")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='user')
        self.formatter.user(f"The question is about the keyword {obs.keyword} in the category {obs.category}. Give yes-or-no answer and surround your answer with double asterisks, like **yes** or **no**.")
        self.formatter.start_model_turn()
    def _parse_response(self, response: str, obs: dict):
        answer = self._parse_keyword(response)
        return 'yes' if 'yes' in answer else 'no'

sp = "You are playing the 20 Questions game. In this game, the Answerer thinks of a keyword and responds to yes-or-no questions by the Questioner. The keyword is a specific person, place, or thing."
fse = [
    "Let's play 20 Questions. You are playing the role of the Questioner. Please ask your first question.",
    "Is it a person?", "**no**",
    "Is is a place?", "**yes**",
    "Is it a country?", "**yes** Now guess the keyword.",
    "**France**", "Correct!",
]
agent = None

def get_agent(name: str):
    global agent
    if agent is None and name == 'questioner':
        agent = GemmaQuestionerAgent(sp=sp, fse=fse,)
    elif agent is None and name == 'answerer':
        agent = GemmaAnswererAgent(sp=sp, fse=fse,)
    assert agent is not None, "Agent not initialized."
    return agent

def agent_fn(obs, cfg):
    if obs.turnType == "ask": response = get_agent('questioner')(obs)
    elif obs.turnType == "guess": response = get_agent('questioner')(obs)
    elif obs.turnType == "answer": response = get_agent('answerer')(obs)
    if response is None or len(response) <= 1: return "yes"
    else: return response
```

</div>
<div class="column-right">

# 日本語訳

```python
%%writefile submission/main.py

import torch, itertools, contextlib
import os, sys, re
from typing import Iterable
from pathlib import Path

# Kaggleエージェントのパスを指定
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    # エージェントのライブラリパスを追加
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib'))
    # ウェイトファイルのパスを指定
    WEIGHTS_PATH = os.path.join(KAGGLE_AGENT_PATH, "gemma/pytorch/2b-it/2")
else:
    # ローカルパスを指定
    sys.path.insert(0, "/kaggle/working/submission/lib")
    WEIGHTS_PATH = "/kaggle/input/gemma/pytorch/2b-it/2"

# gemmaの設定をインポート
from gemma.config import get_config_for_2b
# gemmaのモデルをインポート
from gemma.model import GemmaForCausalLM

class GemmaFormatter:
    _start_token = '<start_of_turn>'
    _end_token = '<end_of_turn>'
    
    def __init__(self, sp: str = None, fse: Iterable = None):
        self._system_prompt = sp
        self._few_shot_examples = fse
        self._turn_user = f"{self._start_token}user\n{{}}{self._end_token}\n"
        self._turn_model = f"{self._start_token}model\n{{}}{self._end_token}\n"
        self.reset()
    
    def __repr__(self):
        return self._state
    
    def user(self, prompt):
        self._state += self._turn_user.format(prompt)
        return self
    
    def model(self, prompt):
        self._state += self._turn_model.format(prompt)
        return self
    
    def start_user_turn(self):
        self._state += f"{self._start_token}user\n"
        return self
    
    def start_model_turn(self):
        self._state += f"{self._start_token}model\n"
        return self
    
    def end_turn(self):
        self._state += f"{self._end_token}\n"
        return self
    
    def reset(self):
        self._state = ""
        if self._system_prompt is not None:
            self.user(self._system_prompt)
        if self._few_shot_examples is not None:
            self.apply_turns(self._few_shot_examples, start_agent='user')
        return self
    
    def apply_turns(self, turns: Iterable, start_agent: str):
        formatters = [self.model, self.user] if start_agent == 'model' else [self.user, self.model]
        formatters = itertools.cycle(formatters)
        for fmt, turn in zip(formatters, turns):
            fmt(turn)
        return self

@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
    torch.set_default_dtype(dtype)
    yield
    torch.set_default_dtype(torch.float)

class GemmaAgent:
    def __init__(self, sp=None, fse=None):
        self._device = xm.xla_device()
        self.formatter = GemmaFormatter(sp=sp, fse=fse)
        print("Initializing model")
        model_config = get_config_for_2b()
        model_config.tokenizer = WEIGHTS_PATH + '/tokenizer.model'
        with _set_default_tensor_type(model_config.get_dtype()):
            model = GemmaForCausalLM(model_config)
            model.load_weights(WEIGHTS_PATH + '/gemma-2b-it.ckpt')
            self.model = model.to(self._device).eval()
    
    def __call__(self, obs, *args):
        self._start_session(obs)
        prompt = str(self.formatter)
        response = self._call_llm(prompt)
        response = self._parse_response(response, obs)
        print(f"{response=}")
        return response
    
    def _start_session(self, obs: dict):
        raise NotImplementedError
    
    def _call_llm(self, prompt, max_nt=40, **sampler_kwargs):
        # サンプリングのパラメータを設定
        if sampler_kwargs is None:
            sampler_kwargs = {'temperature': 0.8, 'top_p': 0.9, 'top_k': 60,}
        # モデルを使ってレスポンスを生成
        response = self.model.generate(
            prompt, device=self._device, output_len=max_nt, **sampler_kwargs,)
        return response

    def _parse_keyword(self, response: str):
        # キーワードをレスポンスから抽出
        match = re.search(r"(?<=\*\*)([^*]+)(?=\*\*)", response)
        if match is None: 
            keyword = ''
        else: 
            keyword = match.group().lower()
        return keyword
    
    def _parse_response(self, response: str, obs: dict):
        raise NotImplementedError

def interleave_unequal(x, y):
    # 2つのリストを交互に結合
    return [item for pair in itertools.zip_longest(x, y) for item in pair if item is not None]

class GemmaQuestionerAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def _start_session(self, obs):
        self.formatter.reset()
        self.formatter.user("Let's play 20 Questions. You are playing the role of the Questioner.")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='model')
        if obs.turnType == 'ask':
            self.formatter.user("Please ask a yes-or-no question.")
        elif obs.turnType == 'guess':
            self.formatter.user("Now guess the keyword. Surround your guess with double asterisks.")
        self.formatter.start_model_turn()
    
    def _parse_response(self, response: str, obs: dict):
        if obs.turnType == 'ask':
            match = re.search(".+?\?", response.replace('*', ''))
            if match is None: 
                question = "Is it a person?"  # 人、場所、物のランダムな選択
            else: 
                question = match.group()
            return question
        elif obs.turnType == 'guess':
            guess = self._parse_keyword(response)
            return guess
        else: 
            raise ValueError("Unknown turn type:", obs.turnType)

class GemmaAnswererAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def _start_session(self, obs):
        self.formatter.reset()
        self.formatter.user(f"Let's play 20 Questions. You are playing the role of the Answerer. The keyword is {obs.keyword} in the category {obs.category}.")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='user')
        self.formatter.user(f"The question is about the keyword {obs.keyword} in the category {obs.category}. Give yes-or-no answer and surround your answer with double asterisks, like **yes** or **no**.")
        self.formatter.start_model_turn()
    
    def _parse_response(self, response: str, obs: dict):
        answer = self._parse_keyword(response)
        return 'yes' if 'yes' in answer else 'no'

# ゲームの設定
sp = "You are playing the 20 Questions game. In this game, the Answerer thinks of a keyword and responds to yes-or-no questions by the Questioner. The keyword is a specific person, place, or thing."
fse = [
    "Let's play 20 Questions. You are playing the role of the Questioner. Please ask your first question.",
    "Is it a person?", "**no**",
    "Is it a place?", "**yes**",
    "Is it a country?", "**yes** Now guess the keyword.",
    "**France**", "Correct!",
]
agent = None

# エージェント取得関数
def get_agent(name: str):
    global agent
    if agent is None and name == 'questioner':
        agent = GemmaQuestionerAgent(sp=sp, fse=fse,)
    elif agent is None and name == 'answerer':
        agent = GemmaAnswererAgent(sp=sp, fse=fse,)
    assert agent is not None, "Agent not initialized."
    return agent

# エージェントの関数
def agent_fn(obs, cfg):
    if obs.turnType == "ask": 
        response = get_agent('questioner')(obs)
    elif obs.turnType == "guess": 
        response = get_agent('questioner')(obs)
    elif obs.turnType == "answer": 
        response = get_agent('answerer')(obs)
    
    if response is None or len(response) <= 1: 
        return "yes"
    else: 
        return response
```

</div>
</details>

In [ ]:
%%writefile submission/main.py

import torch, itertools, contextlib
import os, sys, re
from typing import Iterable
from pathlib import Path

# Kaggleエージェントのパスを指定
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    # エージェントのライブラリパスを追加
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib'))
    # ウェイトファイルのパスを指定
    WEIGHTS_PATH = os.path.join(KAGGLE_AGENT_PATH, "gemma/pytorch/2b-it/2")
else:
    # ローカルパスを指定
    sys.path.insert(0, "/kaggle/working/submission/lib")
    WEIGHTS_PATH = "/kaggle/input/gemma/pytorch/2b-it/2"

# gemmaの設定をインポート
from gemma.config import get_config_for_2b
# gemmaのモデルをインポート
from gemma.model import GemmaForCausalLM

class GemmaFormatter:
    _start_token = '<start_of_turn>'
    _end_token = '<end_of_turn>'
    
    def __init__(self, sp: str = None, fse: Iterable = None):
        self._system_prompt = sp
        self._few_shot_examples = fse
        self._turn_user = f"{self._start_token}user\n{{}}{self._end_token}\n"
        self._turn_model = f"{self._start_token}model\n{{}}{self._end_token}\n"
        self.reset()
    
    def __repr__(self):
        return self._state
    
    def user(self, prompt):
        self._state += self._turn_user.format(prompt)
        return self
    
    def model(self, prompt):
        self._state += self._turn_model.format(prompt)
        return self
    
    def start_user_turn(self):
        self._state += f"{self._start_token}user\n"
        return self
    
    def start_model_turn(self):
        self._state += f"{self._start_token}model\n"
        return self
    
    def end_turn(self):
        self._state += f"{self._end_token}\n"
        return self
    
    def reset(self):
        self._state = ""
        if self._system_prompt is not None:
            self.user(self._system_prompt)
        if self._few_shot_examples is not None:
            self.apply_turns(self._few_shot_examples, start_agent='user')
        return self
    
    def apply_turns(self, turns: Iterable, start_agent: str):
        formatters = [self.model, self.user] if start_agent == 'model' else [self.user, self.model]
        formatters = itertools.cycle(formatters)
        for fmt, turn in zip(formatters, turns):
            fmt(turn)
        return self

@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
    torch.set_default_dtype(dtype)
    yield
    torch.set_default_dtype(torch.float)

class GemmaAgent:
    def __init__(self, sp=None, fse=None):
        self._device = xm.xla_device()
        self.formatter = GemmaFormatter(sp=sp, fse=fse)
        print("Initializing model")
        model_config = get_config_for_2b()
        model_config.tokenizer = WEIGHTS_PATH + '/tokenizer.model'
        with _set_default_tensor_type(model_config.get_dtype()):
            model = GemmaForCausalLM(model_config)
            model.load_weights(WEIGHTS_PATH + '/gemma-2b-it.ckpt')
            self.model = model.to(self._device).eval()
    
    def __call__(self, obs, *args):
        self._start_session(obs)
        prompt = str(self.formatter)
        response = self._call_llm(prompt)
        response = self._parse_response(response, obs)
        print(f"{response=}")
        return response
    
    def _start_session(self, obs: dict):
        raise NotImplementedError
    
    def _call_llm(self, prompt, max_nt=40, **sampler_kwargs):
        # サンプリングのパラメータを設定
        if sampler_kwargs is None:
            sampler_kwargs = {'temperature': 0.8, 'top_p': 0.9, 'top_k': 60,}
        # モデルを使ってレスポンスを生成
        response = self.model.generate(
            prompt, device=self._device, output_len=max_nt, **sampler_kwargs,)
        return response

    def _parse_keyword(self, response: str):
        # キーワードをレスポンスから抽出
        match = re.search(r"(?<=\*\*)([^*]+)(?=\*\*)", response)
        if match is None: 
            keyword = ''
        else: 
            keyword = match.group().lower()
        return keyword
    
    def _parse_response(self, response: str, obs: dict):
        raise NotImplementedError

def interleave_unequal(x, y):
    # 2つのリストを交互に結合
    return [item for pair in itertools.zip_longest(x, y) for item in pair if item is not None]

class GemmaQuestionerAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def _start_session(self, obs):
        self.formatter.reset()
        self.formatter.user("Let's play 20 Questions. You are playing the role of the Questioner.")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='model')
        if obs.turnType == 'ask':
            self.formatter.user("Please ask a yes-or-no question.")
        elif obs.turnType == 'guess':
            self.formatter.user("Now guess the keyword. Surround your guess with double asterisks.")
        self.formatter.start_model_turn()
    
    def _parse_response(self, response: str, obs: dict):
        if obs.turnType == 'ask':
            match = re.search(".+?\?", response.replace('*', ''))
            if match is None: 
                question = "Is it a person?"  # 人、場所、物のランダムな選択
            else: 
                question = match.group()
            return question
        elif obs.turnType == 'guess':
            guess = self._parse_keyword(response)
            return guess
        else: 
            raise ValueError("Unknown turn type:", obs.turnType)

class GemmaAnswererAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def _start_session(self, obs):
        self.formatter.reset()
        self.formatter.user(f"Let's play 20 Questions. You are playing the role of the Answerer. The keyword is {obs.keyword} in the category {obs.category}.")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='user')
        self.formatter.user(f"The question is about the keyword {obs.keyword} in the category {obs.category}. Give yes-or-no answer and surround your answer with double asterisks, like **yes** or **no**.")
        self.formatter.start_model_turn()
    
    def _parse_response(self, response: str, obs: dict):
        answer = self._parse_keyword(response)
        return 'yes' if 'yes' in answer else 'no'

# ゲームの設定
sp = "You are playing the 20 Questions game. In this game, the Answerer thinks of a keyword and responds to yes-or-no questions by the Questioner. The keyword is a specific person, place, or thing."
fse = [
    "Let's play 20 Questions. You are playing the role of the Questioner. Please ask your first question.",
    "Is it a person?", "**no**",
    "Is it a place?", "**yes**",
    "Is it a country?", "**yes** Now guess the keyword.",
    "**France**", "Correct!",
]
agent = None

# エージェント取得関数
def get_agent(name: str):
    global agent
    if agent is None and name == 'questioner':
        agent = GemmaQuestionerAgent(sp=sp, fse=fse,)
    elif agent is None and name == 'answerer':
        agent = GemmaAnswererAgent(sp=sp, fse=fse,)
    assert agent is not None, "Agent not initialized."
    return agent

# エージェントの関数
def agent_fn(obs, cfg):
    if obs.turnType == "ask": 
        response = get_agent('questioner')(obs)
    elif obs.turnType == "guess": 
        response = get_agent('questioner')(obs)
    elif obs.turnType == "answer": 
        response = get_agent('answerer')(obs)
    
    if response is None or len(response) <= 1: 
        return "yes"
    else: 
        return response

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Testing

</div>
<div class="column-right">

# 日本語訳

# テスト


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!pip install -q pygame
```

</div>
<div class="column-right">

# 日本語訳

```python
!pip install -q pygame
```

</div>
</details>

In [ ]:
!pip install -q pygame

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from kaggle_environments import make
env = make("llm_20_questions")
```

</div>
<div class="column-right">

# 日本語訳

```python
from kaggle_environments import make
# Kaggle環境のインスタンスを作成
env = make("llm_20_questions")
```

</div>
</details>

In [ ]:
from kaggle_environments import make
# Kaggle環境のインスタンスを作成
env = make("llm_20_questions")

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
#Run Code
%run submission/main.py
```

</div>
<div class="column-right">

# 日本語訳

```python
# コードを実行
%run submission/main.py
```

</div>
</details>

In [ ]:
# コードを実行
%run submission/main.py

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
#*** Needs to be fixed ****
#env.run([get_agent('questioner'), "random"])
```

</div>
<div class="column-right">

# 日本語訳

```python
#*** 修正が必要 ***
#env.run([get_agent('questioner'), "random"])
```

</div>
</details>

In [ ]:
#*** 修正が必要 ***
#env.run([get_agent('questioner'), "random"])

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
env.render(mode="ipython")
```

</div>
<div class="column-right">

# 日本語訳

```python
# Kaggle環境を表示
env.render(mode="ipython")
```

</div>
</details>

In [ ]:
# Kaggle環境を表示
env.render(mode="ipython")

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Package

</div>
<div class="column-right">

# 日本語訳

# パッケージ


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!apt install pigz pv > /dev/null
```

</div>
<div class="column-right">

# 日本語訳

```python
!apt install pigz pv > /dev/null
```

</div>
</details>

In [ ]:
!apt install pigz pv > /dev/null

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission . -C /kaggle/input/ gemma/pytorch/2b-it/2
```

</div>
<div class="column-right">

# 日本語訳

```python
!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission . -C /kaggle/input/ gemma/pytorch/2b-it/2
```

</div>
</details>

In [ ]:
!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission . -C /kaggle/input/ gemma/pytorch/2b-it/2